In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [ ]:
import pandas as pd, numpy as np 
from travis_attack.config import Config
from travis_attack.insights import get_training_dfs, _prepare_df_concat
from travis_attack.utils import display_all
from IPython.core.debugger import set_trace
import logging 
logger = logging.getLogger("run")

In [ ]:
cfg = Config()
#run_id = 'cdy5d5yu'
run_name = "dazzling-deluge-122"
path_run = f"{cfg.path_checkpoints}{run_name}/"

## Looking at some examples 

Below we just look at some examples to get a feel for what is going on. 

In [ ]:
def get_interesting_idx(df, n):
    def get_idx_with_top_column_values(cname, n=5, ascending=False):
        return df[['idx',cname]].\
            drop_duplicates().\
            sort_values(cname, ascending=ascending)\
            ['idx'][0:n].values.tolist()
    
    def sample_idx_with_label_flips(n=5): 
        df1 = df[['idx','label_flip']].query("label_flip!=0")
        if len(df1) == 0 : print("No label flips detected"); return None
        else: return df1.drop_duplicates()['idx'].sample(n).values.tolist()
    
    idx_d = dict(
        random = df.idx.drop_duplicates().sample(n).tolist(),
        label_flips = sample_idx_with_label_flips(n=n),
        idx_n_unique_pp  = get_idx_with_top_column_values('idx_n_unique_pp',n=n,ascending=False),
       # idx_n_pp_changes = get_idx_with_top_column_values('idx_n_pp_changes',n=n,ascending=False),
        high_contradiction = get_idx_with_top_column_values('contradiction_score',n=n,ascending=False)
    )
    return idx_d

def print_stats(df, idx_d, key, i):
    print("\n###############\n")
    print(key, i, "\n")
    if idx_d[key] is None: return
    idx = idx_d[key][i]
    # Setup 
    df1 = df.query('idx==@idx')
    orig = pd.unique(df1['orig_l'])[0]
    print("Original:", orig)
    print("Original label", pd.unique(df1['orig_label'])[0] )
    pp_all = list(df1['pp_l'])
    #print("All paraphrases", pp_all)
    pp_unique = list(pd.unique(df1['pp_l']))
    n_pp_unique = len(pp_unique)

    # showing a "timeline" of how the paraphrases change over the epochs
    g_fields = ["pp_l","pp_truelabel_probs","vm_score","sts_score","pp_letter_diff", "contradiction_score", "reward","label_flip"]
    #g_fields = ["pp_l","vm_score"]
    g = df1.groupby(g_fields).agg({'epoch' : lambda x: list(x)})
    g = g.sort_values(by='epoch', key = lambda col: col.map(lambda x: np.min(x)))
    print("Unique paraphrases:", n_pp_unique)
    print("How the paraphrases change:")
    display_all(g)

    # Showing a dataframe of the few best paraphrases
    best_pps = df1.sort_values('reward', ascending=False).iloc[0]
    print("Best Paraphrase")
    display_all(best_pps.to_frame().T)
        
def print_interesting_text_stats(df, n): 
    idx_d = get_interesting_idx(df, n)
    for key in idx_d.keys():
        for i in range(n): 
            print_stats(df, idx_d, key,i)

In [ ]:
split = 'valid'
df_d = get_training_dfs(path_run, postprocessed=True)
idx_d = get_interesting_idx(df_d[split], n=5)
print_interesting_text_stats(df_d[split], n=5)


###############

random 0 

Original: a somewhat disappointing and meandering saga .
Original label 0
Unique paraphrases: 5
How the paraphrases change:


,,,,,,,,epoch
pp_l,pp_truelabel_probs,vm_score,sts_score,pp_letter_diff,contradiction_score,reward,label_flip,
The saga was somewhat disappointing.,0.88924,0.04479,0.91521,10,0.000223,0.53746,0,[1]
The saga was a bit disappointing and meandered.,0.93166,0.00237,0.89811,-1,0.000150,0.02841,0,[2]
a saga was done,0.56830,0.36573,0.57980,31,0.418609,0.00000,0,[3]
The saga is a bit meandering,0.87470,0.05933,0.87383,18,0.000792,0.71193,0,[4]
There is a saga but it is meandering,0.89434,0.03969,0.75627,10,0.000302,0.47627,0,[5]


Best Paraphrase


,idx,epoch,orig_l,pp_l,orig_truelabel_probs,pp_truelabel_probs,pp_predclass_probs,orig_label,pp_predclass,label_flip,vm_score,sts_score,pp_letter_diff,pp_letter_percent,contradiction_score,reward,pp_logp,loss,batch_num,global_step,acc_num,loss_sum,loss_batch,label_flip_fraction,orig_length,orig_batch_size,pp_length,pp_batch_size,idx_n_unique_pp,idx_n_pp_changes,epoch_of_first_label_flip,n_words_orig,n_sentences_orig,n_punctuation_orig,n_digits_orig,n_letters_orig,n_words_pp,n_sentences_pp,n_punctuation_pp,n_digits_pp,n_letters_pp,n_words_diff,n_sentences_diff,n_punctuation_diff,n_digits_diff,n_letters_diff,removals_idx,removals,insertions_idx,insertions,unchanged_idx,unchanged,n_segments_inserted,n_segments_removed,n_tokens_inserted,n_tokens_removed,is_truncation,any_phrase_capitalised,any_phrase_decapitalised,edit_distance_token_level
2768,653,4,a somewhat disappointing and meandering saga .,The saga is a bit meandering,0.934028,0.8747,0.874701,0,0,0,0.05933,0.87383,18,0.608696,0.00079163,0.71193,-5.37032,3.82327,13,3724,0,744.908,46.5568,0.238095,16,63,15,63,5,4,1,6,1,1,0,39,6,1,0,0,23,0,0,1,0,16,"[5, 6, 7, 9, 10]","[somewhat disappointing and, saga.]","[0, 1, 2, 4]","[The saga is, bit]","[3, 8]","[a, meandering]",2,2,4,5,True,False,False,7



###############

random 1 

Original: one of those movies that make us pause and think of what we have given up to acquire the fast-paced contemporary society .
Original label 1
Unique paraphrases: 5
How the paraphrases change:


...

,idx,epoch,orig_l,pp_l,orig_truelabel_probs,pp_truelabel_probs,pp_predclass_probs,orig_label,pp_predclass,label_flip,vm_score,sts_score,pp_letter_diff,pp_letter_percent,contradiction_score,reward,pp_logp,loss,batch_num,global_step,acc_num,loss_sum,loss_batch,label_flip_fraction,orig_length,orig_batch_size,pp_length,pp_batch_size,idx_n_unique_pp,idx_n_pp_changes,epoch_of_first_label_flip,n_words_orig,n_sentences_orig,n_punctuation_orig,n_digits_orig,n_letters_orig,n_words_pp,n_sentences_pp,n_punctuation_pp,n_digits_pp,n_letters_pp,n_words_diff,n_sentences_diff,n_punctuation_diff,n_digits_diff,n_letters_diff,removals_idx,removals,insertions_idx,insertions,unchanged_idx,unchanged,n_segments_inserted,n_segments_removed,n_tokens_inserted,n_tokens_removed,is_truncation,any_phrase_capitalised,any_phrase_decapitalised,edit_distance_token_level
667,146,3,"its compelling mix of trial movie , escape movie and unexpected fable ensures the film never feels draggy .","The movie can be trial or escape and if that is what makes the film draggy, it comes from a mix of trial and escape films.",0.939903,0.1737,0.826304,1,0,1,0.76621,0.75607,-15,1.14019,0.173479,3,-40.6905,122.072,8,2793,1,3029.82,275.438,0.328125,24,64,36,64,5,4,3,17,1,2,0,87,26,1,2,0,95,-9,0,0,0,-8,"[10, 13, 28, 29, 31, 32, 33, 34, 35, 36, 37, 40, 41, 42]","[its, compelling, movie,, movie and unexpected fable ensures the film, never feels draggy]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 12, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 27, 38]","[The movie can be trial or escape and if that, is, what makes the film draggy, it comes from a, and, films]","[24, 25, 26, 30, 43]","[mix of trial, escape, .]",5,5,23,14,False,False,False,24



###############

high_contradiction 4 

Original: chelsea walls is a case of too many chefs fussing over too weak a recipe .
Original label 0
Unique paraphrases: 5
How the paraphrases change:


,,,,,,,,epoch
pp_l,pp_truelabel_probs,vm_score,sts_score,pp_letter_diff,contradiction_score,reward,label_flip,
chelsea walls is a case of chefs making a weak recipe and wanting to make it strong.,0.90701,0.03742,0.84067,-10,0.079269,0.44908,0,[1]
"Chefs work to make a stronger recipe without getting too weak, which is what the chelsea walls is.",0.46983,0.47460,0.68335,-25,0.129447,3.00000,1,[2]
"Chefs make too strong of a recipe to impress, so they add chelsea walls as the example",0.91247,0.03196,0.82357,-12,0.998827,0.00000,0,[3]
Chefs tend to over bake weak recipes at chelsea walls because it is a case,0.90168,0.04275,0.85194,0,0.004726,0.51295,0,[4]
Chefs fuss over Weak recipe recipes when chelsea walls but it is a case of if they are trying,0.90326,0.04117,0.83111,-19,0.001043,0.49398,0,[5]


Best Paraphrase


,idx,epoch,orig_l,pp_l,orig_truelabel_probs,pp_truelabel_probs,pp_predclass_probs,orig_label,pp_predclass,label_flip,vm_score,sts_score,pp_letter_diff,pp_letter_percent,contradiction_score,reward,pp_logp,loss,batch_num,global_step,acc_num,loss_sum,loss_batch,label_flip_fraction,orig_length,orig_batch_size,pp_length,pp_batch_size,idx_n_unique_pp,idx_n_pp_changes,epoch_of_first_label_flip,n_words_orig,n_sentences_orig,n_punctuation_orig,n_digits_orig,n_letters_orig,n_words_pp,n_sentences_pp,n_punctuation_pp,n_digits_pp,n_letters_pp,n_words_diff,n_sentences_diff,n_punctuation_diff,n_digits_diff,n_letters_diff,removals_idx,removals,insertions_idx,insertions,unchanged_idx,unchanged,n_segments_inserted,n_segments_removed,n_tokens_inserted,n_tokens_removed,is_truncation,any_phrase_capitalised,any_phrase_decapitalised,edit_distance_token_level
2416,574,2,chelsea walls is a case of too many chefs fussing over too weak a recipe .,"Chefs work to make a stronger recipe without getting too weak, which is what the chelsea walls is.",0.94443,0.46983,0.530171,0,1,1,0.4746,0.68335,-25,1.33784,0.129447,3,-35.8496,107.549,9,1862,0,1459.19,91.1996,0.1875,24,64,31,64,5,4,2,15,1,1,0,58,18,1,2,0,79,-3,0,-1,0,-21,"[20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]",[a case of too many chefs fussing over too weak a recipe],"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]","[Chefs work to make a stronger recipe without getting too weak, which is what the]","[17, 18, 19, 32]","[chelsea walls is, .]",1,1,17,12,False,False,False,18


## Looking at common removals and insertions 

In [ ]:
def get_common_removals_and_insertions(df_concat): 
    idx = df_concat[['data_split','orig_l', 'pp_l']].drop_duplicates().index
    df_unique_pp = df_concat[['data_split','orig_l', 'pp_l','insertions', 'removals']].iloc[idx]
    def flatten_list(l): return [item for sublist in l for item in sublist] 
    removals_flat   =  flatten_list(df_unique_pp['removals'].values)
    insertions_flat =  flatten_list(df_unique_pp['insertions'].values)
    return pd.value_counts(removals_flat), pd.value_counts(insertions_flat)

In [ ]:
df_concat = _prepare_df_concat(df_d)
removals, insertions = get_common_removals_and_insertions(df_concat)

print("\n#### REMOVALS ####\n")
print(removals.head(30))
print("\n#### INSERTIONS ####\n")
print(insertions.head(30))

Here you can look at a specific phrase and examples of where it appears. 

In [ ]:
def investigate_phrase(phrase, cname, n ): 
    mask = [phrase in strs for strs in df_concat[cname]]
    display_all(df_concat[mask].sample(n))

In [ ]:
investigate_phrase('despite', 'removals', 4)